## Bass model written in Tensorflow

The simplest form of the [Bass diffusion model](https://en.wikipedia.org/wiki/Bass_diffusion_model) can be written as:

$f(t) = (p + q \cdot F(t)) \cdot (1-F(t))$

$s(t)  = m \cdot f(t)$

where 

* $f(t)$ is the change of the installed base fraction at time $t$
* $F(t)$ is the installed base fraction at time $t$ (i.e., $F(t) = \sum_{t=1}^{t-1} f(t)$)
* $p$ is the coefficient of innovation
* $q$  is the coefficient of imitation
* $m$ ultimate market potential
* $s(t)$  at time $t$ is the sales at time $t$, i.e., the rate of change of installed base (i.e. adoption) $f(t)$  multiplied by the ultimate market potential $m$.
* $S(t)$ is the total sales at time $t$, equal to $S(t) = m \cdot F(t)$



#### First basic computation

We will now implement a simple graph in Tensorflow to write that computation. We will assume that we know the variables $p$, $q$, $F(t)$, and $m$, and then compute the value of $S(t)$.


In [2]:
import tensorflow as tf
import numpy as np

In [4]:
p = tf.constant(0.02)
q = tf.constant(0.38)
Ft = tf.constant(0.2)
m = tf.constant(25000.)

ft = tf.mul( tf.add(p, tf.mul( q, Ft)) , tf.sub(1., Ft))
st = tf.mul(ft , m)

with tf.Session() as sess:
    result = sess.run(st)
    print(result)


1920.0


Notice that we just implemented a trivially easy numeric operation, that we could also write as a one-liner in Python.

```
St = m * (p + q * Ft) * (1 - Ft)
```

We will see the real value of our approach later, when we will start doing computations with unknown parameters.

#### Fitting the model

[Following the guidelines](http://www.bassbasement.org/BassModel/Default.aspx), we solve the differential equation for $F(t)$:

\begin{equation}
F(t) = \frac{1 - e^{-(p+q)t}} {1+\frac{q}{p}e^{-(p+q)t}}
\end{equation}

Then, we rewrite our code for computing the value $S(t) = m \cdot F(t)$ for a given set of parameters and different values of $t$.

In [7]:
def getSt(p, q, m, t):
    pq = tf.add(p, q)
    pqt = tf.exp( -tf.mul(pq, t))
    Ft = tf.div( tf.sub(1., pqt ),  tf.add(1., tf.mul( tf.div(q, p), pqt ) ) )
    St = tf.mul(Ft , m)
    return St

Now, we will apply the first TensorFlow trick. Instead of computing a single instance of $S(t)$, we will create a vector for the parameter $t$, and get back a vector of instances of $S(t)$.

In [8]:
p = tf.constant(0.001)
q = tf.constant(0.28)
m = tf.constant(46000.)
time = tf.linspace(0., 50, 51)

St = getSt(p, q, m, time)

init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    result = sess.run(St)
    print(result)

[     0.             53.05213547    123.12915039    215.61453247
    337.53466797    498.01623535    708.83813477    985.07324219
   1345.78869629   1814.73376465   2420.87719727   3198.54248047
   4186.78710938   5427.50976562   6961.73242188   8823.64746094
  11032.52050781  13583.50097656  16439.69335938  19528.79882812
  22747.16992188  25972.18945312  29080.234375    31964.96484375
  34550.41796875  36796.03125     38693.90625     40261.203125
  41530.94921875  42543.76953125  43341.6953125   43964.19140625
  44446.1171875   44817.01953125  45101.171875    45318.109375    45483.28125
  45608.7890625   45704.01171875  45776.16796875  45830.80078125
  45872.13671875  45903.390625    45927.01953125  45944.87890625
  45958.3671875   45968.55859375  45976.2578125   45982.07421875
  45986.45703125  45989.77734375]


### Estimating Parameter Values from Data

Now, let's assume that we have the values of $S(t)$ over time, and let's try to estimate a variable.

In [ ]:
# Data
sf_data = np.array( [0., 43., 87., 132., 122., 176., 228., 278., 574., 543., 903., 1129., 1462., 1654., 1848.]).astype(np.float32)
SF_data = np.cumsum(sf_data).astype(np.float32)
print SF_data

Now we are going to setup our model, by defining which of the variables (which we set as constants before) are actually unknown, and should be modeled as random variables.

In [ ]:
# Setup the model
p = tf.Variable(tf.random_uniform([1], 0, 0.1))
# p = tf.constant(0.01)
q = tf.Variable(tf.random_uniform([1], 0, 2.0))
# q = tf.constant(0.38)
m = tf.Variable(tf.random_normal([1], mean=50000, stddev=25000))
# m = tf.constant(50000.)

l = len(SF_data)
time = tf.linspace(0., l, l)
SF = getSFt(p, q, m, time)

In [ ]:
# Minimize the mean squared errors.
loss = tf.reduce_mean( tf.square(SF - SF_data) ) 
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss)

In [ ]:
# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)


In [ ]:
for step in range(100001):
    sess.run(train)
    if step % 20000 == 0:
        print(step, sess.run(p), sess.run(q), sess.run(m))
print(step, sess.run(p), sess.run(q), sess.run(m))